In [6]:
crime_raw_data=pd.read_csv("data/seoul_violent_crime_2019.csv",encoding='euc-kr')
crime_raw_data.head()

,구분,죄종,발생검거,건수
0,중부,살인,발생,1
1,중부,살인,검거,1
2,중부,강도,발생,4
3,중부,강도,검거,4
4,중부,강간,발생,127


In [8]:
#no null data
crime_raw_data= crime_raw_data[crime_raw_data["죄종"].notnull()]
crime_raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 310 entries, 0 to 309
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   구분      310 non-null    object
 1   죄종      310 non-null    object
 2   발생검거    310 non-null    object
 3   건수      310 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 12.1+ KB


In [41]:
#pivot_table
#use pivot_table to reform the dataframe sorted by police station
crime_station = crime_raw_data.pivot_table(
    crime_raw_data, index=["구분"], columns=["죄종","발생검거"], aggfunc=[np.sum])
crime_station.head(2)

sum                                          
       건수                                          
죄종     강간       강도     살인      절도          폭력      
발생검거   검거   발생  검거  발생 검거 발생   검거    발생    검거    발생
구분                                                 
강남    411  496   3   3  2  3  878  2003  1877  2182
강동    155  180  14  15  5  6  907  1650  1873  2161

In [42]:
crime_station.columns

MultiIndex([('sum', '건수', '강간', '검거'),
            ('sum', '건수', '강간', '발생'),
            ('sum', '건수', '강도', '검거'),
            ('sum', '건수', '강도', '발생'),
            ('sum', '건수', '살인', '검거'),
            ('sum', '건수', '살인', '발생'),
            ('sum', '건수', '절도', '검거'),
            ('sum', '건수', '절도', '발생'),
            ('sum', '건수', '폭력', '검거'),
            ('sum', '건수', '폭력', '발생')],
           names=[None, None, '죄종', '발생검거'])

In [43]:
#simplify the dataframe's columns
crime_station.columns = crime_station.columns.droplevel([0,1])

In [44]:
crime_station.columns

MultiIndex([('강간', '검거'),
            ('강간', '발생'),
            ('강도', '검거'),
            ('강도', '발생'),
            ('살인', '검거'),
            ('살인', '발생'),
            ('절도', '검거'),
            ('절도', '발생'),
            ('폭력', '검거'),
            ('폭력', '발생')],
           names=['죄종', '발생검거'])

In [38]:
crime_station.head(1)

죄종     강간      강도    살인      절도          폭력      
발생검거   검거   발생 검거 발생 검거 발생   검거    발생    검거    발생
구분                                               
강남    411  496  3  3  2  3  878  2003  1877  2182

In [16]:
#fame sorted by burglar and arrest
crime_station["강도","검거"].head()

구분
강남     3
강동    14
강북     2
강서     4
관악    11
Name: (강도, 검거), dtype: int64

In [46]:
#fame sorted by murder and number
crime_station["살인","발생"].head()

구분
강남     3
강동     6
강북     6
강서     8
관악    13
Name: (살인, 발생), dtype: int64

In [48]:
# inlining the columns simply  
tmp = crime_station.columns.get_level_values(0)+crime_station.columns.get_level_values(1)
tmp

Index(['강간검거', '강간발생', '강도검거', '강도발생', '살인검거', '살인발생', '절도검거', '절도발생', '폭력검거',
       '폭력발생'],
      dtype='object')

In [50]:
crime_station.columns = tmp
crime_station.head()

,강간검거,강간발생,강도검거,강도발생,살인검거,살인발생,절도검거,절도발생,폭력검거,폭력발생
구분,,,,,,,,,,
강남,411,496,3,3,2,3,878,2003,1877,2182
강동,155,180,14,15,5,6,907,1650,1873,2161
강북,145,176,2,3,6,6,664,1229,2204,2422
강서,257,306,4,3,9,8,1077,2050,2192,2555
관악,335,408,11,10,10,13,1085,2223,2363,2667


### 1. how often have the viorent crimes occurred in each area?

In [51]:
# index is consisting of stations
crime_station.index

Index(['강남', '강동', '강북', '강서', '관악', '광진', '구로', '금천', '남대문', '노원', '도봉',
       '동대문', '동작', '마포', '방배', '서대문', '서부', '서초', '성동', '성북', '송파', '수서',
       '양천', '영등포', '용산', '은평', '종로', '종암', '중랑', '중부', '혜화'],
      dtype='object', name='구분')

In [56]:
#not all the stations represent the area where these are located in
#match the stations with its areas  
station_name=["해화","남대문","중부","종암","서부","방배","수서"]
gu_name = ["종로구","중구","중구","성북구","은평구","서초구","강남구"]

In [57]:
tmp_gu=[]
for idx, _ in crime_station.iterrows():
    if idx in station_name:
        tmp_gu.append(gu_name[station_name.index(idx)])
    else:
        tmp_gu.append(idx+"구")

In [63]:
#add new area column
crime_station["구"]= tmp_gu
crime_station.head(2)

,강간검거,강간발생,강도검거,강도발생,살인검거,살인발생,절도검거,절도발생,폭력검거,폭력발생,구
구분,,,,,,,,,,,
강남,411,496,3,3,2,3,878,2003,1877,2182,강남구
강동,155,180,14,15,5,6,907,1650,1873,2161,강동구


In [86]:
#new dataframe sorted by area
crime_gu = pd.pivot_table(crime_station, index="구", aggfunc=np.sum)
crime_gu.head()

,강간검거,강간발생,강도검거,강도발생,살인검거,살인발생,절도검거,절도발생,폭력검거,폭력발생
구,,,,,,,,,,
강남구,562,666,6,5,3,5,1339,2970,3153,3652
강동구,155,180,14,15,5,6,907,1650,1873,2161
강북구,145,176,2,3,6,6,664,1229,2204,2422
강서구,257,306,4,3,9,8,1077,2050,2192,2555
관악구,335,408,11,10,10,13,1085,2223,2363,2667


In [87]:
#muder
crime_gu["살인발생"].sort_values(ascending=False).head()

구
관악구     13
구로구     10
중랑구     10
영등포구    10
강서구      8
Name: 살인발생, dtype: int64

In [88]:
#assault
crime_gu["폭력발생"].sort_values(ascending=False).head()

구
강남구     3652
영등포구    3039
송파구     2989
관악구     2667
서초구     2636
Name: 폭력발생, dtype: int64

In [89]:
#crime arrest rates columns on dataframe
arrest=["강간검거","강도검거","살인검거","절도검거","폭력검거"]
occurs=["강간발생","강도발생","살인발생","절도발생","폭력발생"]
new_columns=["강간검거율","강도검거율","살인검거율","절도검거율","폭력검거율"]
crime_gu[new_columns]=crime_gu[occurs].div(crime_gu[arrest].values)*100
crime_gu.head()

,강간검거,강간발생,강도검거,강도발생,살인검거,살인발생,절도검거,절도발생,폭력검거,폭력발생,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율
구,,,,,,,,,,,,,,,
강남구,562,666,6,5,3,5,1339,2970,3153,3652,118.505338,83.333333,166.666667,221.807319,115.826197
강동구,155,180,14,15,5,6,907,1650,1873,2161,116.129032,107.142857,120.000000,181.918412,115.376401
강북구,145,176,2,3,6,6,664,1229,2204,2422,121.379310,150.000000,100.000000,185.090361,109.891107
강서구,257,306,4,3,9,8,1077,2050,2192,2555,119.066148,75.000000,88.888889,190.343547,116.560219
관악구,335,408,11,10,10,13,1085,2223,2363,2667,121.791045,90.909091,130.000000,204.884793,112.865002


In [90]:
crime_gu = crime_gu.drop(columns=arrest)

In [92]:
crime_gu[crime_gu[new_columns]>100]=100

In [105]:
crime_gu.head()

,강간,강도,살인,절도,폭력,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율
구,,,,,,,,,,
강남구,666,5,5,2970,3652,100.0,83.333333,100.000000,100.0,100.0
강동구,180,15,6,1650,2161,100.0,100.000000,100.000000,100.0,100.0
강북구,176,3,6,1229,2422,100.0,100.000000,100.000000,100.0,100.0
강서구,306,3,8,2050,2555,100.0,75.000000,88.888889,100.0,100.0
관악구,408,10,13,2223,2667,100.0,90.909091,100.000000,100.0,100.0


In [106]:
#rename columns 
crime_gu.rename(columns={
    "강간발생" : "강간",
    "강도발생" : "강도",
    "살인발생" : "살인",
    "절도발생" : "절도",
    "폭력발생" : "폭력"},
    inplace = True)

In [107]:
crime_gu.head()

,강간,강도,살인,절도,폭력,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율
구,,,,,,,,,,
강남구,666,5,5,2970,3652,100.0,83.333333,100.000000,100.0,100.0
강동구,180,15,6,1650,2161,100.0,100.000000,100.000000,100.0,100.0
강북구,176,3,6,1229,2422,100.0,100.000000,100.000000,100.0,100.0
강서구,306,3,8,2050,2555,100.0,75.000000,88.888889,100.0,100.0
관악구,408,10,13,2223,2667,100.0,90.909091,100.000000,100.0,100.0


In [109]:
#normalization 
col=["살인","강도","강간","절도","폭력"]
col2= ["강간검거율","강도검거율","살인검거율","절도검거율","폭력검거율"]

crime_gu_norm = crime_gu[col] / crime_gu[col].max()
crime_gu_norm[col2]= crime_gu[col2]
crime_gu_norm.head()

,살인,강도,강간,절도,폭력,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율
구,,,,,,,,,,
강남구,0.384615,0.333333,1.000000,1.000000,1.000000,100.0,83.333333,100.000000,100.0,100.0
강동구,0.461538,1.000000,0.270270,0.555556,0.591731,100.0,100.000000,100.000000,100.0,100.0
강북구,0.461538,0.200000,0.264264,0.413805,0.663198,100.0,100.000000,100.000000,100.0,100.0
강서구,0.615385,0.200000,0.459459,0.690236,0.699617,100.0,75.000000,88.888889,100.0,100.0
관악구,1.000000,0.666667,0.612613,0.748485,0.730285,100.0,90.909091,100.000000,100.0,100.0


In [111]:
crime_gu_norm["범죄"] = np.mean(crime_gu_norm[col],axis=1)
crime_gu_norm.head()

,살인,강도,강간,절도,폭력,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율,범죄
구,,,,,,,,,,,
강남구,0.384615,0.333333,1.000000,1.000000,1.000000,100.0,83.333333,100.000000,100.0,100.0,0.743590
강동구,0.461538,1.000000,0.270270,0.555556,0.591731,100.0,100.000000,100.000000,100.0,100.0,0.575819
강북구,0.461538,0.200000,0.264264,0.413805,0.663198,100.0,100.000000,100.000000,100.0,100.0,0.400561
강서구,0.615385,0.200000,0.459459,0.690236,0.699617,100.0,75.000000,88.888889,100.0,100.0,0.532939
관악구,1.000000,0.666667,0.612613,0.748485,0.730285,100.0,90.909091,100.000000,100.0,100.0,0.751610


### visualization

In [112]:

x = np.linspace(0, 14,100)
y1 = np.sin(x)
y2= 2*np.sin(x)
